[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hendersonneurolab/CogAI_Fall2025/blob/master/Lab06_fMRI_Intro.ipynb)

## Week 6: Working with fMRI data

In this tutorial, we'll learn how to work with functional MRI data. We'll load some example data, and make some visualizations. We will also learn how to construct a general linear model (GLM) and compute beta weights for voxels.

Learning Objectives:
- Gain familiarity with the format and dimensions of functional MRI data, and how to work with this data.
- Understand the hemodynamic response function, and how it affects the timecourse in fMRI data.
- Know how to use a simple t-test to find active voxels in an fMRI experiment.



Credit: some of this code and example data was originally generated by [Leila Wehbe](https://www.cs.cmu.edu/~lwehbe/).

In [ ]:
import numpy as np
import urllib.request
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import scipy

First, let's download some example data that we can work with.

In [ ]:
# data example
dbox_link1 = 'https://www.dropbox.com/scl/fi/1irp9n4zmjmuociha8dqa/example_data_01.npz?rlkey=sbupp8wdhce4a86c6we9qo1fy&st=f5f787f1&dl=1'

# mask
dbox_link2 = 'https://www.dropbox.com/scl/fi/p26wbp5hmjy9d07ibximz/s01_mask.npy?rlkey=ot0mxlpda68fxv2lyvivt8fzn&st=kif9minu&dl=1'

# experiment design file
dbox_link3 = 'https://www.dropbox.com/scl/fi/i8drkfb2ra8f8n7j3pw03/experiment_design.npz?rlkey=igwqu8ldtus0m7eg8xprm5u0k&st=idzruvfd&dl=1'

# data file
dbox_link4 = 'https://www.dropbox.com/scl/fi/087etb9idyt9equu9x6an/sub01_categories1.npy?rlkey=qfmozcqryt2fd0yta1cq8w86m&st=04sfwc4m&dl=1'


In [ ]:
url = dbox_link1
with urllib.request.urlopen(url) as response:
    buffer = BytesIO(response.read())
    ex_data = np.load(buffer)

url = dbox_link2
with urllib.request.urlopen(url) as response:
    buffer = BytesIO(response.read())
    mask = np.load(buffer)

url = dbox_link3
with urllib.request.urlopen(url) as response:
    buffer = BytesIO(response.read())
    design = np.load(buffer)

url = dbox_link4
with urllib.request.urlopen(url) as response:
    buffer = BytesIO(response.read())
    data = np.load(buffer)
    data = data.astype('float32')
    data = data.T

In [ ]:
print('mask.shape : ', mask.shape)
print('data.shape : ', data.shape)
print('design.keys() : ', design.keys())

In [ ]:
# Set the TR, or repetition time, a.k.a. the sampling rate
TR = 2.0 # one measurement every 2 seconds

### Part 1: Visualizing the data.

Let's visualize the data in slice form.
For now, we will look at just the first timepoint. We can specify an x, y, z coordinate for the slices.

- x = left/right axis (makes a sagittal slice)
- y = back/front axis (makes a coronal slice)
- z = up/down axis (makes a horizontal slice)

Units are in "percent signal change" (notice how large the numbers are).

In [ ]:

slice_xyz = [15, 50, 40]

plt.figure(figsize=(14, 6))

my_slice = data[0, :, :, slice_xyz[2]]

plt.subplot(1,3,1)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.title('Sagittal Slice (x=%d)'%slice_xyz[0])
plt.gca().invert_yaxis()
plt.colorbar()

my_slice = data[0, :, slice_xyz[1], :]

plt.subplot(1,3,2)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.gca().invert_yaxis()
plt.title('Coronal Slice (y=%d)'%slice_xyz[1])
plt.colorbar()

my_slice = data[0, slice_xyz[0], :, :]

plt.subplot(1,3,3)
plt.imshow(my_slice, cmap='viridis', aspect=1)
plt.title('Horizontal Slice (z=%d)'%slice_xyz[2])
plt.colorbar()

Make some plotting helper functions here.

In [ ]:
def get_any_slice(volume, slice_number, dimension):
    """Given an integer and a 3D volume, this function returns the data of
    that horizontal slice """
    if dimension == 0:
        img = volume[slice_number, :, :]
    elif dimension == 1:
        img = volume[:, slice_number, :]
    elif dimension == 2:
        img = volume[:, :, slice_number]
    return img

def plot_any_slice(volume, slice_number, dimension, origin = 'lower', interpolation='nearest', aspect='equal',
                cmap='viridis', vmin=0, vmax=2000):
    img = get_any_slice(volume, slice_number, dimension)
    if dimension>0:
      aspect=2
    _ = plt.imshow(img, origin = origin, interpolation=interpolation, aspect=aspect,
                cmap=cmap, vmin=vmin, vmax=vmax)

    _ = plt.axis('off')


def plot_all_slices(volume, slice_dimension, nrows, ncols , cmap = 'viridis', vmin=0, vmax=2000,
                     origin = 'lower', interpolation='nearest', aspect='equal' ):
    fig = plt.figure(figsize = (12, 12))
    n_slices = volume.shape[slice_dimension]
    for s in range(n_slices):
        ax = fig.add_subplot(nrows, ncols, s+1)
        plot_any_slice(volume, s, slice_dimension, cmap = cmap, vmin= vmin, vmax = vmax,
                          origin = origin, interpolation = interpolation,aspect = aspect)



Now visualize all the slices. This gives us a sense of the overall brain anatomy.

In [ ]:
plot_all_slices(data[0], 0, 5, 6)
plt.suptitle('Horizontal slices');

In [ ]:
plot_all_slices(data[0], 2, 10, 10)
plt.suptitle('Sagittal slices');

In [ ]:
plot_all_slices(data[0], 1, 10, 10)
plt.suptitle('Coronal slices');

Make a histogram of the values in the data. Here, we're plotting all the values from the above plots.

In [ ]:
# Make a histogram of the data
print(data.shape)
plt.figure()
plt.hist(data.flatten(), bins=50)
plt.show()



---
***Question 1:***

What does the distribution of the data look like? Why are there so many zeros?

[answer here]



---



So far, we've been working with all the voxels in the brain. For most analyses, we actually want to work with just the voxels in the cortex. We loaded a mask earlier that has all our cortical voxels labeled. Let's inspect it now.

In [ ]:
# Display the same information as the brain mask above
cortical_voxels =mask
print(cortical_voxels.dtype)
print(cortical_voxels.shape)
print(cortical_voxels.sum())
print(cortical_voxels.mean())

There are 38543 voxels in the cortical mask. Let's focus on these for our remaining analysis.

In [ ]:
print('Raw data shape: ')
print(data.shape)
print('Mask shape: ')
print(mask.shape)
data_masked = data[:,mask]
data_masked.shape
print('Masked data shape: ')
print(data_masked.shape)

In [ ]:
# Make a histogram of the data
# data_mask
print(data_masked.shape)
plt.figure()
plt.hist(data_masked.flatten(), bins=50)
plt.show()



---
***Question 2:***

Now that we have applied the mask, what does the distribution of the data look like? How is this different from the raw data?

[answer here]



---



Now let's plot a few individual voxels. We can see that their distributions are different, some have larger signal than others.

In [ ]:
vv1 = 0; vv2 = 2; vv3 = 10
voxel_a = data_masked[:,vv1]
voxel_b = data_masked[:,vv2]
voxel_c = data_masked[:,vv3]
print('Mean A: %.2f'%np.mean(voxel_a))
print('Mean B: %.2f'%np.mean(voxel_b))
print('Mean C: %.2f'%np.mean(voxel_c))
_ = plt.hist(voxel_a, label='Voxel A')
_ = plt.hist(voxel_b, label='Voxel B')
_ = plt.hist(voxel_c, label='Voxel C')
plt.legend()

In general, the mean differences between voxels are not something we care about that much, for our analyses. We care a lot about the *differences* in response for individual voxels, as in their relative responses to different experimental conditions. In order to isolate these relative differences, we typically will z-score each voxel's response before we proceed with our analyses.

In [ ]:
data_z_masked = np.nan_to_num(scipy.stats.zscore(data, axis=0))[:,mask]

Let's plot how the z-scoring changed things...

In [ ]:
vv1 = 0; vv2 = 2; vv3 = 10
voxel_a = data_z_masked[:,vv1]
voxel_b = data_z_masked[:,vv2]
voxel_c = data_z_masked[:,vv3]
print('Mean A: %.2f'%np.mean(voxel_a))
print('Mean B: %.2f'%np.mean(voxel_b))
print('Mean C: %.2f'%np.mean(voxel_c))
_ = plt.hist(voxel_a, label='Voxel A')
_ = plt.hist(voxel_b, label='Voxel B')
_ = plt.hist(voxel_c, label='Voxel C')
plt.legend()

And here's another way to view it. In this plot, each column is a voxel, and each row is a timepoint.
We can see that in the raw data, most of the variance is across voxels (vertical lines). But in the z-scored data, we have removed this across-voxel variance, so we can see more of the across-time variance.

In [ ]:
plt.figure(figsize=(12, 4))
plt.pcolormesh(data_masked)
plt.xlabel('Voxels')
plt.ylabel('TRs')
plt.colorbar()
plt.title('Raw data')

plt.figure(figsize=(12, 4))
plt.pcolormesh(data_z_masked)
plt.xlabel('Voxels')
plt.ylabel('TRs')
plt.colorbar()
plt.title('Z-Scored')

Now let's plot a few voxel timecourses, and see how the data look.

In [ ]:
vox2plot = [500, 1000, 1200]

for vv in vox2plot:
  plt.figure(figsize=(12, 4))
  plt.plot(data_z_masked[:,vv])

  plt.xlabel('TRs')
  plt.ylabel('fMRI activity')
  plt.title('Voxel number %d'%vv)

### Part 2: Analyze the data with a general linear model (GLM).

First, let's visualize the experiment design. This is an experiment that includes responses to images in 5 different categories. There are blocks of time where the participant is shown images in the same category, lasting 20 seconds each. Hence, it is a *block design.*

Between the blocks, there is a period where no images is shown, which allows us to estimate the "baseline" signal.

Typically, an experiment like this will be used to localize regions of interest (ROIs) that are selective for each category.

There are 3 runs, and each run has 120 TRs.

In this context a "run" refers to one length of time over which fMRI data is continuously collected. Usually they are a few minutes long, and there can be many runs per session.

In [ ]:
print('Experiment design variables: ', design.keys())
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)

Visualize the structure on each run. Note that the first few seconds have been trimmed off, so the run starts with some condition already "on".

In [ ]:
plt.figure(figsize=(12, 4))

for rr, run in enumerate(['run1','run2','run3']):

  plt.subplot(1,3,rr+1)
  for i in range(5):
    evt_on = design[run][:,i]
    cond = evt_on
    plt.plot(cond+i+0.2*i, label = conditions[i], lw=2)
    # plt.plot(time_axis, cond, label = conditions[i], lw=2)
  plt.xlabel('TRs')
  plt.title('Run %d'%rr)
  _ = plt.legend(frameon=False, bbox_to_anchor=(1.2, 1))



We can also concatenate the runs and view it as one big experiment:

In [ ]:
design_all_runs = np.vstack([design[r] for r in ['run1','run2','run3']])
plt.figure(figsize=(12,4))
for i, (cond, label) in enumerate(zip(design_all_runs.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
    plt.xlabel('TR number')
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.2, 1))

plt.axvline(120, color=[0.8, 0.8, 0.8], zorder=-10)
plt.axvline(240, color=[0.8, 0.8, 0.8], zorder=-10)



---
***Question 3:***

If the TR length is 2 seconds. How many total seconds of data do we have, across these 3 runs?

In [ ]:
# [answer here]



---



Using this experimental design information, we can now create our general linear model. We will be modeling each voxel's response as a sum of predictors, which correspond to the different conditions (image categories).

For this to work well, we also need to account for the delay in each voxel's response, by modeling the **hemodynamic response function (HRF).**

The HRF is a function that captures the timecourse of fMRI response (Blood-oxygen-level-dependent / BOLD response) following a burst of neural activity. From past research, we know the approximate shape of this function, and can model it using a gamma distribution.

Let's first create a function that makes a canonical HRF function.


In [ ]:
def generate_hrf(shape='twogamma', tr=1, pttp=5, nttp=15, pos_neg_ratio=6, onset=0, pdsp=1, ndsp=1, t=None):
    # can alter these parameters to change the shape.
    if t is None:
        t = np.arange(0, (onset + 2 * (nttp + 1)), tr) - onset
    else:
        t = np.arange(np.min(t), np.max(t), tr) - onset;

    # Create filter
    h = np.zeros((len(t), ))
    if shape.lower()=='boynton':
        # boynton (single-gamma) HRF
        h = scipy.stats.gamma.pdf(t, pttp + 1, pdsp)
    elif shape.lower()=='twogamma':
        gpos = scipy.stats.gamma.pdf(t, pttp + 1, pdsp)
        gneg = scipy.stats.gamma.pdf(t, nttp + 1, ndsp) / pos_neg_ratio
        h =  gpos-gneg
    h /= np.sum(h)
    return t, h

In [ ]:
# Get a canonical HRF
tr = 2
t_hrf, hrf_1 = generate_hrf(shape='twogamma', tr=tr, \
                            pttp=5, nttp=15, \
                            pos_neg_ratio=6, onset=0, \
                            pdsp=1, ndsp=1, t=None)
print('hrf_1 shape is', hrf_1.shape)

plt.figure(figsize=(4,3))
__ = plt.plot(t_hrf, hrf_1, '.-')
plt.title("Canonical HRF")
plt.xlabel("Time (s)")
plt.ylabel("Normalized amplitude")



---
***Question 4:***

Make a new code block, and create new versions of the HRF. Change the parameters in the generate_hrf function. Try changing: *pttp*, *pos_neg_ratio*, and the other parameters. Try setting "shape" to "boynton".

What does each parameter do to the HRF shape?

In [ ]:
#[answer here]



---



Now we'll use convolution to predict the response of a voxel given our proposed HRF function.

We can start with a simulation. We will create a timeseries of simulated event onsets. For example, these could represent image onsets. Each event onset is modeled as a "delta function". This is a function with zeros everywhere, but 1 at the event time. It represents an infinitely short event. In theory, if the neural activity in the brain is captured by a delta function, then the fMRI response to it will look exactly like the HRF. If there are several events, we would see several HRF-like responses.

If we **convolve** our timeseries of delta functions with the HRF, we can get a prediction of what a voxel response should look like.

This predicted response is what we expect to get for an ideal voxel that is perfectly responsive to the simulated events, and has zero noise in its responses.

Note that this form of convolution is not unlike what we have seen earlier in the class, with convolutional neural networks (CNNs). Recall how in a 2D CNN, we took some 2D filter, and we slid it across a 2D image, taking the dot product at each location to get our feature map responses. This is a similar idea, only now it's in 1D only. The HRF is like the filter, and we are sliding it over the event timeseries to get the predicted response.

In [ ]:
# how many total timepoints to simulate?
n_timepts = 360
# this is where the events happen
evt_onsets = [50, 80, 200]

time_axis = np.arange(n_timepts,)

# now we're creating simulated event sequence
evt_times = np.zeros((n_timepts,))
for ee in evt_onsets:
  evt_times[ee] = 1

# now we convolve event times with HRF (np.convolve)
# make sure to trim off the last timepoints, which go outside event sequence
convolved_resp = np.convolve(evt_times, hrf_1, mode = 'full')[0:n_timepts]

ylims = [-0.5,1.2]

# and plot
plt.figure(figsize=(8, 3))
plt.plot(time_axis, evt_times, '.-', color='k')
plt.title('Events')
plt.ylim(ylims)
plt.xlim([-5, n_timepts+5])
plt.vlines(evt_onsets, ymin = ylims[0], ymax=ylims[1], color=[0.8, 0.8, 0.8], zorder=-10)

plt.figure(figsize=(8, 3))
plt.plot(time_axis, convolved_resp)
plt.title('Convolved response (predicted BOLD)')
plt.ylim(ylims)
plt.xlim([-5, n_timepts+5])
plt.vlines(evt_onsets, ymin = ylims[0], ymax=ylims[1], color=[0.8, 0.8, 0.8], zorder=-10)




---
***Question 5:***

Try making your own simulated series of events (start by copying some of the code above), and see how the sequence changes. What happens if you make several events very close together?


In [ ]:
# [answer here]



---



Now we will apply this same idea to the actual events in our data sequence. We have a series of conditions and the timing of when they were presented. We'll convolve each condition sequence with the HRF, to generate convolved predictors.

In [ ]:
D = design_all_runs
t_hrf, hrf_1 = generate_hrf(tr=2)
n, d = D.shape

conv_D = np.zeros_like(D)
for i in range(d):
    conv_D[:,i] = np.convolve(D[:,i], hrf_1)[:n]


plt.figure(figsize=(12,4))
for i, (cond, label) in enumerate(zip(D.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)

plt.title('Original predictors')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))
plt.axvline(120, color=[0.8, 0.8, 0.8], zorder=-10)
plt.axvline(240, color=[0.8, 0.8, 0.8], zorder=-10)


plt.figure(figsize=(12,4))
for i, (cond, label) in enumerate(zip(conv_D.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)

plt.title('Convolved predictors')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))
plt.axvline(120, color=[0.8, 0.8, 0.8], zorder=-10)
plt.axvline(240, color=[0.8, 0.8, 0.8], zorder=-10)

For each voxel, we will model it as a weighted sum of the predictors above (i.e., the colored lines). We are assuming that the voxel is linearly related to each of the experimental conditions.

We write the linear model equation as:

\begin{align}
Y^{v} =  {\bf X} W^v +\epsilon^v
\end{align}

Since this model exist for every function, we can write it as a multiple regression function:
\begin{align}
{\bf Y} =  {\bf X} {\bf W} +\boldsymbol\epsilon
\end{align}

In the above:
- ${\bf Y}$ is [nTRs x nVoxels]
- ${\bf X}$ is [nTRs x nConditions]
- ${\bf W}$ is [nConditions x nVoxels]
- $\epsilon$ is [nTRs x nVoxels]

We want to solve for ${\bf W}$, which contains our **beta weights**. In this case we have 5 conditions, so we will get 5 beta weights per voxel: one for each condition.

To solve this, we can use ordinary least squares (OLS). The solution will be:

\begin{align}
{{\bf W} = ({\bf X}^\top{\bf X})^{-1}{\bf X}^\top {\bf Y}\\}
\end{align}

Notice also that each voxel's parameters are estimated independently from each other: each column of ${\bf W}$ corresponds to the parameters of one voxel $v$, and it is obtained by multipling the matrix $({\bf X}^\top{\bf X})^{-1}{\bf X}^\top$ with the ${\bf Y}$ column that corresponds to voxel $v$.


In [ ]:
# Compute W (beta weights)

X = conv_D
Y = data_z_masked

XtX_inv = np.linalg.inv(np.dot(X.T, X))
beta = np.dot(XtX_inv, np.dot(X.T, Y))

print('betas shape: ')
print(beta.shape)

Now compute some other metrics.

We can convert our beta weights into t-statistics, which reflect the estimate of each parameter normalized by the standard error.

$$t = \frac{\hat{\beta}}{SE(\hat{\beta})}$$

We can also compute $R^2$, or the proportion of variance explained. $R^2$ is a measure of overall fit quality.

$$R^2 = \frac{\text{variance explained by model}}{\text{total variance}}$$



In [ ]:

# Compute residuals
Y_pred = np.dot(X, beta)
residuals = Y - Y_pred

# Degrees of freedom
n = X.shape[0]  # number of observations
p = X.shape[1]  # number of parameters
df = n - p

# Residual sum of squares
RSS = np.sum(residuals**2, axis=0)

# Residual variance (MSE)
sigma2 = RSS / df

# Standard errors of beta
# SE(beta) = sqrt(sigma^2 * diag(XtX_inv))
var_beta = sigma2 * np.diag(XtX_inv)[:, np.newaxis]  # Shape: (p, 1) or (p, n_voxels)
se_beta = np.sqrt(var_beta)

# t-statistics
t_stats = beta / se_beta

# Compute R^2 (variance explained)
TSS = np.sum((Y - np.mean(Y, axis=0))**2, axis=0)
R2 = 1 - RSS / TSS



---
***Question 6:***

Print out the minimum, mean, and maximum values of $R^2$.

In [ ]:
# [answer here]



---



Plot an example timecourse, and see how good the fit is.

***Note:*** this fitting was all done using the same data for training and testing.
In practice, we'd really want to evaluate R2 on an independent test set. These values may be inflated, due to being computed on the same data.

In [ ]:
good_vox = np.flip(np.argsort(R2))[0:1]
# I'm choosing the best voxel for this - not all will be so good

for vv in good_vox:

  plt.figure()
  plt.plot(Y[:,vv])
  plt.plot(Y_pred[:,vv])

  plt.title('voxel %d, R2 = %.2f'%(vv, R2[vv]))

  plt.legend(['Actual', 'Predicted'], loc='upper left')

Now let's put the voxels back into volumetric space, so we can see where the best R2 is.

These plots are showing a heatmap of R2 values.

In [ ]:

mask_vals = cortical_voxels.copy().astype(np.float32)
mask_vals[mask_vals>0] = R2
mask_vals[mask_vals==0] = np.nan

# can change these coords to get slightly different brain regions.
slice_xyz = [15, 50, 40]

plt.figure(figsize=(14, 6))

my_slice = mask_vals[:, :, slice_xyz[2]]

plt.subplot(1,3,1)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.title('Sagittal Slice (x=%d)'%slice_xyz[0])
plt.gca().invert_yaxis()
plt.colorbar()

my_slice = mask_vals[:, slice_xyz[1], :]

plt.subplot(1,3,2)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.gca().invert_yaxis()
plt.title('Coronal Slice (y=%d)'%slice_xyz[1])
plt.colorbar()

my_slice = mask_vals[slice_xyz[0], :, :]

plt.subplot(1,3,3)
plt.imshow(my_slice, cmap='viridis', aspect=1)
plt.title('Horizontal Slice (z=%d)'%slice_xyz[2])
plt.colorbar()

plt.suptitle('R2 values')




---
***Question 7:***

Which parts of the brain have the best $R^2$ values? Does this make sense?


[answer here]



---



Finally, let's compare the activity between conditions, using a **contrast**. This is similar to a t-test.

We will create a vector over our conditions, which specifies the comparison to perform.

Let's start by contrasting face selectivity versus place selectivity. For this, we put a 1 for faces, a -1 for places, and a 0 for everything else. Basically, this means you are taking the beta weight for faces minus the beta weight for places.

The variable "conditions" tells us what the conditions are:

In [ ]:
conditions

In [ ]:
# this defines what contrast / test we want to run.
contrast_vector = np.array([0, 1, 0, -1, 0])

In [ ]:
contrast_est = np.dot(contrast_vector.T, beta)  # Shape: (1, n_voxels)

# Compute standard error of contrast: SE(c' * beta) = sqrt(sigma^2 * c' * (X'X)^-1 * c)
var_contrast = sigma2 * np.dot(np.dot(contrast_vector.T, XtX_inv), contrast_vector)  # Shape: (1, n_voxels)
se_contrast = np.sqrt(var_contrast)

# Compute t-statistic for contrast
t_stats_contrast = contrast_est / se_contrast
t_stats_contrast = t_stats_contrast.squeeze()  # Shape: (n_voxels,)

print(f"\nContrast t-statistics shape: {t_stats_contrast.shape}")
print(f"Mean contrast t-stat: {np.mean(t_stats_contrast):.2f}")
print(f"Number of voxels with |t| > 2: {np.sum(np.abs(t_stats_contrast) > 2)}")

Now plot this in volumetric space.
Notice that we can see a stripe of yellow (positive t) and dark blue (negative t) along on the ventral surface, in the sagittal view. This corresponds to face-selective and scene-selective regions.

In [ ]:

mask_vals = cortical_voxels.copy().astype(np.float32)
mask_vals[mask_vals>0] = t_stats_contrast
mask_vals[mask_vals==0] = np.nan

# can change these coords to get slightly different brain regions.
slice_xyz = [15, 50, 40]

plt.figure(figsize=(14, 6))

my_slice = mask_vals[:, :, slice_xyz[2]]

plt.subplot(1,3,1)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.title('Sagittal Slice (x=%d)'%slice_xyz[0])
plt.gca().invert_yaxis()
plt.colorbar()

my_slice = mask_vals[:, slice_xyz[1], :]

plt.subplot(1,3,2)
plt.imshow(my_slice, cmap='viridis', aspect=2)
plt.gca().invert_yaxis()
plt.title('Coronal Slice (y=%d)'%slice_xyz[1])
plt.colorbar()

my_slice = mask_vals[slice_xyz[0], :, :]

plt.subplot(1,3,3)
plt.imshow(my_slice, cmap='viridis', aspect=1)
plt.title('Horizontal Slice (z=%d)'%slice_xyz[2])
plt.colorbar()

plt.suptitle('T-Statistics')




---
***Question 8:***

Copy the code above, and modify it to do a contrast between [object] > [scrambled]. What do the results look like? Which areas are highlighted?

*Bonus:* do you know the names of the areas this contrast can be used to define?


[answer here]



---

